In [4]:
#configure btata he k job kese run krni he 

%%configure
{
  "--enable-spark-ui": "true",
  "--spark-event-logs-path": "s3://sufis-bucket/output/lab3/sparklog/",
  "max_retries": "0"
}


The following configurations have been updated: {'--enable-spark-ui': 'true', '--spark-event-logs-path': 's3://sufis-bucket/output/lab3/sparklog/', 'max_retries': '0'}


In [1]:
# Importing Glue and Spark libraries
import sys #In Glue jobs, it’s used when reading parameters passed at job runtime
from awsglue.transforms import * #This imports Glue’s built-in data transformation helpers.
from awsglue.utils import getResolvedOptions # Special Glue helper for reading parameters you pass when starting a Glue job
from pyspark.context import SparkContext #starts the Spark engine.
from awsglue.context import GlueContext # adds Glue features on top of Spark.
from awsglue.job import Job #tells Glue you’re running a Glue job.
from awsglue.dynamicframe import DynamicFrame #Glue’s special table format (more flexible than DataFrame).


import boto3 #lets you call AWS services from Python
from pprint import pprint # prints results in a nice readable way.

from datetime import datetime #works with timestamps.

# Start Spark/Glue Context
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)


Trying to create a Glue session for the kernel.
Session Type: glueetl
Session ID: fe8b7c03-6823-4c60-b162-e82b97a02975
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true
--enable-spark-ui true
--spark-event-logs-path s3://sufis-bucket/output/lab3/sparklog/
--max_retries 0
Waiting for session fe8b7c03-6823-4c60-b162-e82b97a02975 to get into ready status...
Session fe8b7c03-6823-4c60-b162-e82b97a02975 has been created.



In [6]:
from pyspark.context import SparkContext 
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame

# Glue context
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

# Glue Catalog details
catalog_db = " pgdatabase-1"       # Your Glue catalog database name
catalog_table = "sampledata"      # Your Glue catalog table name

# PostgreSQL connection details
rds_host = "postgres.clwcyeea822i.eu-north-1.rds.amazonaws.com"
rds_port = "5432"
rds_db = "postgres"
rds_user = "postgres"  # Replace with your actual RDS username
rds_password = "Paki$tani123$$$"
target_table = "sampledata_target"


# Read from Glue Catalog table
dyf = glueContext.create_dynamic_frame.from_catalog(
    database=catalog_db,
    table_name=catalog_table
)
#DynamicFrame = AWS Glue’s friendly format for messy data.

#DataFrame = Spark’s strict format for clean data and analytics.

# Convert DynamicFrame -- DataFrame
df = dyf.toDF()

df.show(5)
df.printSchema()

# JDBC URL (JDBC 1 driver he jo k talk to any database provide krta he )

jdbc_url = f"jdbc:postgresql://{rds_host}:{rds_port}/{rds_db}"


# Write to RDS PostgreSQL
df.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", target_table) \
    .option("user", rds_user) \
    .option("password", rds_password) \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()

print(" Data successfully loaded into PostgreSQL RDS from Glue Catalog table")


+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|  Ali| 25|
|  2|Ahmed| 30|
|  3| Sara| 28|
+---+-----+---+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

✅ Data successfully loaded into PostgreSQL RDS from Glue Catalog table
